In [2]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2025-04-02 06:37:12--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8001::154, 2606:50c0:8000::154, 2606:50c0:8002::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8001::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.1’

minsearch.py.1      100%[===================>]   3.74K  --.-KB/s    in 0.001s  

2025-04-02 06:37:13 (6.45 MB/s) - ‘minsearch.py.1’ saved [3832/3832]



In [3]:
import minsearch

In [4]:
import json

In [5]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [6]:
documents = []

for course_dict in docs_raw:
    # print(course_dict)
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [7]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

SELECT * WHERE course = 'data-engineering-zoomcamp';

In [8]:
q = 'the course has already started, can I still enroll?'

In [9]:
index.fit(documents)

# mistral 

In [11]:
import os
from mistralai import Mistral

api_key = os.getenv("MISTRAL_API_KEY")
model = "mistral-large-latest"

client = Mistral(api_key=api_key)

chat_response = client.chat.complete(
    model= model,
    messages = [
        {
            "role": "user",
            "content": "What is the best French cheese?",
        },
    ]
)
print(chat_response.choices[0].message.content)

Determining the "best" French cheese can be highly subjective, as it depends on personal taste preferences. France is renowned for its diverse and rich cheese culture, with over 1,600 different types of cheese. Here are a few highly regarded French cheeses across various categories:

1. **Soft Cheeses**:
   - **Brie de Meaux**: Known for its creamy interior and bloomy rind.
   - **Camembert de Normandie**: A famous soft, creamy cheese with a distinctive aroma.

2. **Semi-Soft Cheeses**:
   - **Reblochon**: A savory cheese from the Alps, often used in tartiflette.
   - **Morbier**: Recognizable by its layer of ash in the middle.

3. **Hard Cheeses**:
   - **Comté**: A fruity and complex cheese made from unpasteurized cow's milk.
   - **Beaufort**: A firm, raw cow's milk cheese with a strong, nutty flavor.

4. **Blue Cheeses**:
   - **Roquefort**: A tangy, crumbly blue cheese made from sheep's milk.
   - **Bleu d'Auvergne**: A strong, creamy blue cheese from the Auvergne region.

5. **Go

In [30]:
from openai import OpenAI
from dotenv import load_dotenv
import os

# Load variables from .env file
load_dotenv()

# Read the key from the environment
api_key = os.getenv("OPENAI_API_KEY")

# Initialize the client
client = OpenAI(api_key=api_key)

In [31]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content



"Whether you can still enroll in a course that has already started depends on the specific policies of the institution offering the course. Here are some steps you can take to find out:\n\n1. **Check the Institution's Policy:** Visit the course or institution's website to see if they mention late enrollment policies or deadlines.\n\n2. **Contact the Instructor or Admissions Office:** Reach out directly to the course instructor or the admissions office to inquire about the possibility of late enrollment. They may be able to provide exceptions or let you know if it's feasible to catch up.\n\n3. **Consider the Course Format:** If the course is offered online or allows for self-paced learning, there might be more flexibility in joining after the start date.\n\n4. **Assess the Impact:** Consider whether starting late will impact your ability to keep up with coursework and participate effectively. You might need to commit extra time initially to catch up.\n\n5. **Consider Payment or Fees:** 

In [32]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [33]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [34]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [35]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [36]:
rag(query)

'To run Kafka, you can execute the following command in the project directory:\n\n```\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```'

In [20]:
# # mistral llmprompt
# def llm(prompt):
#     response = client.chat.completions.create(
#         model="mistral-large-latest",
#         messages=[{"role": "user", "content": prompt}]
#     )
    
#     return response.choices[0].message.content

def run_mistral(user_message, model="mistral-large-latest"):
    messages = [
        {
            "role": "user", "content": user_message
        }
    ]
    chat_response = client.chat.complete(
        model=model,
        messages=messages
    )
    return (chat_response.choices[0].message.content)

# run_mistral(prompt)

In [21]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    # answer = llm(prompt)
    answer = run_mistral(prompt)
    return answer

In [22]:
rag(query)

'To run Kafka, you can follow these instructions based on the context provided:\n\nFor Java Kafka, to run a producer/consumer/kstreams/etc in the terminal, navigate to the project directory and execute:\n```\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nFor Python Kafka, if you encounter the "Module \'kafka\' not found" error when trying to run `producer.py`, follow these steps to create a virtual environment and run the Python files:\n\n1. Create a virtual environment:\n   ```\n   python -m venv env\n   ```\n\n2. Activate the virtual environment:\n   - On MacOS and Linux:\n     ```\n     source env/bin/activate\n     ```\n   - On Windows:\n     ```\n     .\\env\\Scripts\\activate\n     ```\n\n3. Install the required packages:\n   ```\n   pip install -r ../requirements.txt\n   ```\n\n4. Ensure Docker images are up and running before executing the Python file.\n\n5. Deactivate the virtual environment when done:\n   ```\n   deact

In [16]:
rag(query)

'To run Kafka using Java in the terminal, navigate to the project directory and execute the following command:\n\n```bash\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nReplace `<jar_name>` with the actual name of your jar file.'

In [17]:
rag('the course has already started, can I still enroll?')

"Yes, you can still enroll in the course after it has started. Even if you don't register, you are eligible to submit the homeworks. However, be mindful of the deadlines for the final projects and try not to leave everything for the last minute."

In [18]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [19]:
from elasticsearch import Elasticsearch

In [20]:
es_client = Elasticsearch('http://localhost:9200') 

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

In [22]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [23]:
from tqdm.auto import tqdm

/Users/mayurnawal/anaconda3/envs/llm_zoomcamp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

100%|██████████| 948/948 [00:01<00:00, 611.18it/s]


In [25]:
query = 'I just disovered the course. Can I still join it?'

In [26]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [27]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [28]:
rag(query)

"Yes, you can still join the course even after the start date. You are eligible to submit the homework even if you don't register. Just be aware that there will be deadlines for turning in the final projects, so it is advisable not to leave everything for the last minute."

In [ ]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [24]:
pip install elasticsearch

Note: you may need to restart the kernel to use updated packages.


In [26]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc) 

/Users/mayurnawal/anaconda3/envs/llm_zoomcamp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 948/948 [00:01<00:00, 510.85it/s]


In [ ]:
# homework - lecture - 1 

In [47]:
def elastic_search_asn(query):
    search_query = {
        "size": 3, # 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp" # "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    result_score= [] 
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
        result_score.append(hit['_score'])

    
    return result_docs, result_score

In [48]:
def rag(query):
    search_results, result_score = elastic_search_asn(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer, result_score

In [49]:
query = "How do I execute a command in a running docker container?"
rag(query) #  [75.54128, 43.922554, 38.684105, 38.33403, 35.94081])


('To execute a command in a running Docker container, you can use the `docker exec` command. Follow these steps:\n\n1. Find the container ID of the running container using the `docker ps` command.\n2. Use the following command to execute a command inside the container:\n\n```\ndocker exec -it <container-id> <command>\n```\n\nFor example, to start a bash shell in the container, you would use:\n\n```\ndocker exec -it <container-id> bash\n```',
 [84.050095, 51.04628, 49.938507])

In [50]:
queries  = ["How do I debug a docker container?", 
"How do I copy files from a different folder into docker container’s working directory?", 
"How do Lambda container images work?", 
"How can I annotate a graph?"]
for query in queries: 
    rag(query)

In [51]:
rag("How do I debug a docker container?")

('To debug a Docker container, you should launch the container image in interactive mode and override the entrypoint so that it starts a bash command. This can be done using the following command:\n\n```bash\ndocker run -it --entrypoint bash <image>\n```\n\nIf the container is already running and you want to execute a command in that specific container, you can do so by:\n\n1. Finding the container ID:\n   ```bash\n   docker ps\n   ```\n\n2. Executing a bash command inside the running container:\n   ```bash\n   docker exec -it <container-id> bash\n   ```',
 [103.06507, 51.04628, 43.277237])

In [52]:
rag("How do I copy files from a different folder into docker container’s working directory?")

('You can copy files from a different folder into a Docker container’s working directory using the `docker cp` command. The basic syntax for this command is:\n```bash\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\n```\nThis command allows you to copy a file or directory from your local machine into the specified path within a running Docker container.',
 [183.81017, 79.590385, 55.26839])

In [ ]:
rag("How do Lambda container images work?")
# ('Lambda container images work by allowing you to package your code and dependencies as a container image, which can then be used to create AWS Lambda functions. For a detailed understanding of how Lambda container images function and how Lambda functions are initialized, you can refer to the following documentation:\n\n1. [AWS Lambda Images Create Documentation](https://docs.aws.amazon.com/lambda/latest/dg/images-create.html)\n2. [AWS Lambda Runtimes API Documentation](https://docs.aws.amazon.com/lambda/latest/dg/runtimes-api.html)',


('Lambda container images work by allowing you to package your code and dependencies as a container image, which can then be used to create AWS Lambda functions. For a detailed understanding of how Lambda container images function and how Lambda functions are initialized, you can refer to the following documentation:\n\n1. [AWS Lambda Images Create Documentation](https://docs.aws.amazon.com/lambda/latest/dg/images-create.html)\n2. [AWS Lambda Runtimes API Documentation](https://docs.aws.amazon.com/lambda/latest/dg/runtimes-api.html)',
 [114.34481, 44.687473, 41.297256])

In [63]:
def build_prompt_asn(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"question: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [64]:
def rag_prompt(query):
    search_results, result_score = elastic_search_asn(query)
    prompt = build_prompt_asn(query, search_results)
    answer = llm(prompt)
    return answer, result_score, prompt

In [65]:
query = "How do Lambda container images work?"
res = rag_prompt(query) 
res


('Lambda container images allow you to package custom runtimes and dependencies for AWS Lambda functions. This approach gives you more control over the execution environment of your functions. For more in-depth information about how Lambda container images work and how Lambda functions are initialized, you can refer to the following documentation:\n\n- [Creating Lambda container images](https://docs.aws.amazon.com/lambda/latest/dg/images-create.html)\n- [Lambda runtimes API](https://docs.aws.amazon.com/lambda/latest/dg/runtimes-api.html)',
 [114.34481, 44.687473, 41.297256],
 "You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\nUse only the facts from the CONTEXT when answering the QUESTION.\n\nQUESTION: How do Lambda container images work?\n\nCONTEXT: \nquestion: How do Lambda container images work?\nanswer: I wanted to understand how lambda container images work in depth and how lambda functions are initialized, for this reason, I foun

In [66]:
print(len(res[2]))

1237


In [70]:
import tiktoken 
encoding = tiktoken.encoding_for_model("gpt-4o")

In [ ]:
tokens = encoding.encode(res[0])
print(len(tokens)) # prompt token - 293 , response token 103 

103


In [ ]:
# 150 tokens/request × 1000 requests = 150,000 tokens
# 250 tokens/request × 1000 requests = 250,000 tokens
# cost input - (150,000 / 1000) × $0.005 = 150 × 0.005 = $0.75
# cost output - (250,000 / 1000) × $0.015 = 250 × 0.015 = $3.75
# $0.75 (input) + $3.75 (output) = $4.50
